# Yelp Restaurant Recommender System - Part 1: Data Cleaning

In [1]:
import numpy as np
import pandas as pd
import json
from sklearn.preprocessing import MultiLabelBinarizer

%matplotlib inline

In [2]:
# Load JSON files
with open('../yelp_dataset/yelp_academic_dataset_business.json') as json_file:      
    data = json_file.readlines()
    data = list(map(json.loads, data)) 

business = pd.DataFrame(data)

In [3]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ..."
2,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,45.511907,-122.613693,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Antiques, Fashion, Used, Vintage & Consignment...","{'Thursday': '11:0-18:0', 'Friday': '11:0-18:0..."
3,oaepsyvc0J17qwi8cfrOWg,Great Clips,2566 Enterprise Rd,Orange City,FL,32763,28.914482,-81.295979,3.0,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Beauty & Spas, Hair Salons",None
4,PE9uqAjdw0E4-8mjGl3wVA,Crossfit Terminus,1046 Memorial Dr SE,Atlanta,GA,30316,33.747027,-84.353424,4.0,14,1,"{'GoodForKids': 'False', 'BusinessParking': '{...","Gyms, Active Life, Interval Training Gyms, Fit...","{'Monday': '16:0-19:0', 'Tuesday': '16:0-19:0'..."


In [4]:
business['state'].value_counts(ascending=False).head(10)

MA    36012
OR    25175
TX    24485
FL    21907
GA    18090
BC    17298
OH    11258
CO     3198
WA     3121
CA       13
Name: state, dtype: int64

In [5]:
# Check for nulls
business.isnull().sum()

business_id         0
name                0
address             0
city                0
state               0
postal_code         0
latitude            0
longitude           0
stars               0
review_count        0
is_open             0
attributes      14992
categories        115
hours           27341
dtype: int64

In [6]:
# Filter the business dataframe to state of MA 
business = business.loc[business['state'] == 'MA']

# Filter the MA business dataframe to restaurants businesses only
business = business.dropna(axis=0, subset=['categories']) # Drop nulls for categories column
business = business.loc[business['categories'].str.contains('Restaurants')]

In [7]:
# Check for nulls, and drop nulls if necessary, check the shape of filtered dataframe
business.isnull().sum(), business.shape

(business_id        0
 name               0
 address            0
 city               0
 state              0
 postal_code        0
 latitude           0
 longitude          0
 stars              0
 review_count       0
 is_open            0
 attributes        70
 categories         0
 hours           1849
 dtype: int64,
 (10550, 14))

In [8]:
# Drop nulls
business.dropna(axis=0, subset=['attributes'], inplace=True)
business.drop(columns=['hours'], inplace=True)

In [9]:
# Edit column names
business.rename(columns={"stars":"business_stars"}, inplace=True)

In [10]:
# Unpack attributes and categories column
attributes = business['attributes'].apply(pd.Series)
attributes.isnull().sum().sort_values()

RestaurantsTakeOut              510
BusinessAcceptsCreditCards      511
BusinessParking                 711
RestaurantsDelivery             727
RestaurantsPriceRange2          931
OutdoorSeating                 1146
RestaurantsReservations        1301
RestaurantsGoodForGroups       1517
GoodForKids                    1610
Alcohol                        1697
RestaurantsAttire              1823
HasTV                          2010
Ambience                       2046
WiFi                           2536
NoiseLevel                     2856
Caters                         3210
BikeParking                    3294
GoodForMeal                    4835
RestaurantsTableService        6856
WheelchairAccessible           7952
HappyHour                      8459
DogsAllowed                    8593
BYOBCorkage                    8862
BusinessAcceptsBitcoin         9117
Music                          9291
Corkage                        9339
BestNights                     9539
GoodForDancing              

In [11]:
# Remove features from the attributes dataframe
attributes.dropna(axis=1, thresh=5000, inplace=True) # drop attribute columns with 50% nulls
attributes.drop(columns=['Ambience', 'BusinessParking', 'GoodForMeal'], inplace=True) 
# Impute the nulls and get_dummies to binarize the columns
attributes = pd.get_dummies(attributes.fillna('None'))

In [12]:
attributes

,RestaurantsGoodForGroups_False,RestaurantsGoodForGroups_None,RestaurantsGoodForGroups_True,HasTV_False,HasTV_None,HasTV_True,GoodForKids_False,GoodForKids_None,GoodForKids_True,RestaurantsTakeOut_False,...,Caters_False,Caters_None,Caters_True,WiFi_'free',WiFi_'no',WiFi_'paid',WiFi_None,WiFi_u'free',WiFi_u'no',WiFi_u'paid'
12,0,0,1,0,0,1,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
26,0,0,1,1,0,0,0,0,1,0,...,0,0,1,1,0,0,0,0,0,0
29,1,0,0,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
35,0,0,1,1,0,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0
54,0,0,1,0,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160532,0,0,1,0,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
160542,0,0,1,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0
160556,1,0,0,1,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
160563,1,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0


In [39]:
# Filter the attributes column, only keep columns of interest
attributes = attributes[['RestaurantsGoodForGroups_False',
       'RestaurantsGoodForGroups_True', 
       'HasTV_False', 'HasTV_True', 
       'GoodForKids_False', 'GoodForKids_True',
       'RestaurantsTakeOut_False', 'RestaurantsTakeOut_True',
       'RestaurantsPriceRange2_1','RestaurantsPriceRange2_2', 'RestaurantsPriceRange2_3', 'RestaurantsPriceRange2_4', 
       'BikeParking_False', 'BikeParking_True',
       'RestaurantsReservations_False','RestaurantsReservations_True',
       "RestaurantsAttire_'casual'","RestaurantsAttire_'dressy'",  "RestaurantsAttire_u'casual'","RestaurantsAttire_u'dressy'", "RestaurantsAttire_u'formal'",
       'RestaurantsDelivery_False', 'RestaurantsDelivery_True', 
       'OutdoorSeating_False', 'OutdoorSeating_True',
       "NoiseLevel_'average'", "NoiseLevel_'loud'", "NoiseLevel_'quiet'", "NoiseLevel_'very_loud'", "NoiseLevel_u'average'", "NoiseLevel_u'loud'", "NoiseLevel_u'quiet'", "NoiseLevel_u'very_loud'",
       "Alcohol_'beer_and_wine'", "Alcohol_'full_bar'", "Alcohol_u'beer_and_wine'","Alcohol_u'full_bar'", "Alcohol_u'none'",
       'BusinessAcceptsCreditCards_False', 'BusinessAcceptsCreditCards_True',
       'Caters_False', 'Caters_True',
       "WiFi_'free'", "WiFi_'no'", "WiFi_'paid'",  "WiFi_u'free'", "WiFi_u'no'", "WiFi_u'paid'"]]

In [40]:
# Join with business dataframe, and remove old 'attributes' column
business = business.join(attributes)
business.drop(columns='attributes', axis=1, inplace=True)

In [41]:
# Unpack categories column
# Binarize the categories column for business dataframe
mlb = MultiLabelBinarizer()
categories_df = pd.DataFrame(mlb.fit_transform(business['categories'].str.split(',')),columns=mlb.classes_, index=business.index)
# Check for nulls
categories_df.isnull().sum().sort_values()

 Acai Bowls           0
Bed & Breakfast       0
Beer                  0
Beer Bar              0
Beer Gardens          0
                     ..
 Kebab                0
 Keys & Locksmiths    0
 Kids Activities      0
 Japanese             0
Yelp Events           0
Length: 697, dtype: int64

In [42]:
# Join categories sub_dataframe with business dataframe, remove old 'categories' column
business = business.join(categories_df)
business.drop(columns='categories', inplace=True)

In [43]:
# Save and export the cleaned business dataset as a csv file 
business.to_csv('../business.csv')

In [44]:
# Load yelp reviews dataset
with open('../yelp_dataset/yelp_academic_dataset_review.json') as json_file:      
    data = json_file.readlines()
    # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects. 
    data = list(map(json.loads, data)) 

reviews = pd.DataFrame(data)

In [45]:
reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,lWC-xP3rd6obsecCYsGZRg,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4.0,3,1,1,Apparently Prides Osteria had a rough summer a...,2014-10-11 03:34:02
1,8bFej1QE5LXp4O05qjGqXA,YoVfDbnISlW0f7abNQACIg,RA4V8pr014UyUbDvI-LW2A,4.0,1,0,0,This store is pretty good. Not as great as Wal...,2015-07-03 20:38:25
2,NDhkzczKjLshODbqDoNLSg,eC5evKn1TWDyHCyQAwguUw,_sS2LBIGNT5NQb6PD1Vtjw,5.0,0,0,0,I called WVM on the recommendation of a couple...,2013-05-28 20:38:06
3,T5fAqjjFooT4V0OeZyuk1w,SFQ1jcnGguO0LYWnbbftAA,0AzLzHfOJgL7ROwhdww2ew,2.0,1,1,1,I've stayed at many Marriott and Renaissance M...,2010-01-08 02:29:15
4,sjm_uUcQVxab_EeLCqsYLg,0kA0PAJ8QFMeveQWHFqz2A,8zehGz9jnxPqXtOc7KaJxA,4.0,0,0,0,The food is always great here. The service fro...,2011-07-28 18:05:01


In [46]:
# Keep columns with interest
reviews = reviews[['user_id','business_id', 'stars', 'date']]

In [47]:
# Check for nulls
reviews.isnull().sum()

user_id        0
business_id    0
stars          0
date           0
dtype: int64

In [49]:
# Change date column to pd datetime format
reviews['date'] = pd.to_datetime(reviews['date']) # Change 'date' column to 'datetime' format

In [50]:
# Filter reviews dataframe into MA restaurants reviews only
common_business = list(business['business_id'])
reviews = reviews.loc[reviews['business_id'].isin(common_business)]
reviews.rename(columns={'stars': 'review_stars'}, inplace=True)

In [51]:
reviews

,user_id,business_id,review_stars,date
0,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4.0,2014-10-11 03:34:02
5,RNm_RWkcd02Li2mKPRe7Eg,xGXzsc-hzam-VArK6eTvtw,1.0,2018-01-21 04:41:03
6,Q8c91v7luItVB0cMFF_mRA,EXOsmAB1s71WePlQk0WZrA,2.0,2006-04-16 02:58:44
9,99RsBrARhhx60UnAC4yDoA,EEHhKSxUvJkoPSzeGKkpVg,5.0,2014-05-07 18:10:21
18,DECuRZwkUw8ELQZfNGef2Q,zmZ3HkVCeZPBefJJxzdJ7A,4.0,2012-12-04 04:29:47
...,...,...,...,...
8635366,XAAqXanALw3NYFUMw0TYGA,H-NwqS6zuF1zzJ4wyUe3cg,5.0,2020-12-23 20:34:41
8635381,9U30DANobkrn3Zlu6T9p6Q,biYgFkftCPue7g3upflUOg,4.0,2012-01-04 01:40:36
8635388,1WGPvc_cDXt-IPVNqg5BOA,FxveeHL_B0Kkz1KjPKyF3A,5.0,2021-01-18 02:55:18
8635395,fYkURme6Piqxu4qUjQV3PQ,gEQxTJDoJYaW0l_6FYtf8g,5.0,2020-12-05 21:32:45


In [52]:
# Save and export the review dataset as a csv file
reviews.to_csv('../reviews.csv')

In [55]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,business_stars,review_count,...,Venues & Event Spaces,Vietnamese,Waffles,Wedding Planning,Weight Loss Centers,Whiskey Bars,Wine Bars,Wineries,Wraps,Yelp Events
12,HPA_qyMEddpAEtFof02ixg,Mr G's Pizza & Subs,474 Lowell St,Peabody,MA,01960,42.541155,-70.973438,4.0,39,...,0,0,0,0,0,0,0,0,0,0
26,hcRxdDg7DYryCxCoI8ySQA,Longwood Galleria,340-350 Longwood Ave,Boston,MA,02215,42.338544,-71.106842,2.5,24,...,0,0,0,0,0,0,0,0,0,0
29,jGennaZUr2MsJyRhijNBfA,Legal Sea Foods,1 Harborside Dr,Boston,MA,02128,42.363442,-71.025781,3.5,856,...,0,0,0,0,0,0,0,0,0,0
35,iPD8BBvea6YldQZPHzVrSQ,Espresso Minute,334 Mass Ave,Boston,MA,02115,42.342673,-71.084239,4.5,7,...,0,0,0,0,0,0,0,0,0,0
54,Z2JC3Yrz82kyS86zEVJG5A,Gigi's Roast Beef & Pizza,5 Center St,Burlington,MA,01803,42.506935,-71.195854,3.0,16,...,0,0,0,0,0,0,0,0,0,0


In [54]:
reviews.head()

,user_id,business_id,review_stars,date
0,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4.0,2014-10-11 03:34:02
5,RNm_RWkcd02Li2mKPRe7Eg,xGXzsc-hzam-VArK6eTvtw,1.0,2018-01-21 04:41:03
6,Q8c91v7luItVB0cMFF_mRA,EXOsmAB1s71WePlQk0WZrA,2.0,2006-04-16 02:58:44
9,99RsBrARhhx60UnAC4yDoA,EEHhKSxUvJkoPSzeGKkpVg,5.0,2014-05-07 18:10:21
18,DECuRZwkUw8ELQZfNGef2Q,zmZ3HkVCeZPBefJJxzdJ7A,4.0,2012-12-04 04:29:47
